### Import libraries

##### 234 manually labeled tweets, these will be used for training and testing

In [73]:
import numpy as np
import pandas as pd
import scipy as sp   
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.cross_validation import train_test_split #how does this work?
from sklearn.model_selection import train_test_split as tts

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
neighbors = 5
from sklearn.naive_bayes import MultinomialNB
from nltk.classify import NaiveBayesClassifier  #how does this work?

from sklearn.model_selection import cross_val_score as cvs

In [107]:
kanye2 = pd.read_csv('kwest.csv')
kanye2.head()

,text,label
0,RT @WSHHRAP: Rappers with the most Billboard H...,1.0
1,"Kanye and Cudi merge, together they rule as Kanye",1.0
2,RT @RappersSaid: Rappers with the most Billboa...,1.0
3,RT @c_drew_: Kanye shaped like he played barit...,1.0
4,RT @Bubbabue: @kanyewest Your album #ye is an ...,1.0


In [108]:
kanye2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2064 entries, 0 to 2063
Data columns (total 2 columns):
text     2064 non-null object
label    234 non-null float64
dtypes: float64(1), object(1)
memory usage: 32.3+ KB


In [109]:
df = kanye2.sort_values(by = 'label', ascending=True)
df1 = df.reset_index()
df1[233:].head()

,index,text,label
233,116,RT @c_drew_: Kanye shaped like he played barit...,1.0
234,234,RT @ComplexMusic: Drake receives co-writing cr...,NaN
235,235,play only one by kanye at my wedding pls.,NaN
236,236,RT @50cent: Wait we can post shit like this? 👀...,NaN
237,237,RT @nataliesurely: Kim can you please do singl...,NaN


In [110]:
kanye2train = df1[(df1['label'] ==0)|(df1['label']==1)]
kanye2train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 234 entries, 0 to 233
Data columns (total 3 columns):
index    234 non-null int64
text     234 non-null object
label    234 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 7.3+ KB


In [111]:
kanye2train.head()

,index,text,label
0,233,RT @Parkour_Lewis: Cause he slavery shamed our...,0.0
1,155,RT @taydrug: Taylor was dragged for saying she...,0.0
2,156,RT @taydrug: Taylor was dragged for saying she...,0.0
3,157,RT @Parkour_Lewis: Cause he slavery shamed our...,0.0
4,158,RT @Pink: Kanye west is the biggest piece of s...,0.0


In [112]:
kanye2test = df1[df1['label'].isnull()][:50]
kanye2test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 234 to 283
Data columns (total 3 columns):
index    50 non-null int64
text     50 non-null object
label    0 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ KB


In [113]:
kanye2test.head()

,index,text,label
234,234,RT @ComplexMusic: Drake receives co-writing cr...,NaN
235,235,play only one by kanye at my wedding pls.,NaN
236,236,RT @50cent: Wait we can post shit like this? 👀...,NaN
237,237,RT @nataliesurely: Kim can you please do singl...,NaN
238,238,RT @YeezusClips: This is what we got the last ...,NaN


In [114]:
text = kanye2train['text']
print(type(text))
text.head()

<class 'pandas.core.series.Series'>


0    RT @Parkour_Lewis: Cause he slavery shamed our...
1    RT @taydrug: Taylor was dragged for saying she...
2    RT @taydrug: Taylor was dragged for saying she...
3    RT @Parkour_Lewis: Cause he slavery shamed our...
4    RT @Pink: Kanye west is the biggest piece of s...
Name: text, dtype: object

In [115]:
toadd = kanye2test['text']
toadd

234    RT @ComplexMusic: Drake receives co-writing cr...
235            play only one by kanye at my wedding pls.
236    RT @50cent: Wait we can post shit like this? 👀...
237    RT @nataliesurely: Kim can you please do singl...
238    RT @YeezusClips: This is what we got the last ...
239    I need someone who can edit to put Bebe Zahara...
240    When???!!!! Lmao yall b On SuperXtra LMMMAoo h...
241    @jafs31 @jennychoinews Ha ha - as long as I'm ...
242    RT @Complex: Drake receives co-writing credit ...
243    RT @Mitch98Mm: @mastaace Wrd! #Nobody  Sad day...
244    RT @Complex: Drake receives co-writing credit ...
245    Haven’t really listened to Kanye’s album until...
246    RT @YeezusClips: This is what we got the last ...
247    RT @new_branches: Drake wrote the hook for Kan...
248    @BHOPDaMaestro by Lil Baby ft. Drake &amp; Kan...
249    I wanna here a song with Kanye and Wale !! Jus...
250    I’ve never heard Kanye West talk until celebri...
251    i haven't listen to the 

### Let's try TF-IDF

In [116]:
tfidf = text.append(toadd, ignore_index = True)
tfidf.head()

0    RT @Parkour_Lewis: Cause he slavery shamed our...
1    RT @taydrug: Taylor was dragged for saying she...
2    RT @taydrug: Taylor was dragged for saying she...
3    RT @Parkour_Lewis: Cause he slavery shamed our...
4    RT @Pink: Kanye west is the biggest piece of s...
Name: text, dtype: object

In [117]:
vect = TfidfVectorizer(stop_words='english')

In [118]:
tf = pd.DataFrame(vect.fit_transform(tfidf).toarray(), columns=vect.get_feature_names())

In [119]:
tf.head()

,100,1063atl,10s,11,12,148lqlf5ao,16,17,18,18f4qrgqlx,...,yikes,ylpcvbcqvn,ynxi6p3c7u,youtube,yxotobksfb,zahara,zmh9qhh6xk,znu3q5mi58,zth5gym7xn,zuqwdrzfdd
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
tf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284 entries, 0 to 283
Columns: 860 entries, 100 to zuqwdrzfdd
dtypes: float64(860)
memory usage: 1.9 MB


In [121]:
reset = kanye2train['label'].reset_index()
del reset ['index']
reset.head()

,label
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [122]:
kanye = pd.concat([tf,reset], axis = 1,ignore_index= False)
kanye['label'][233:].head()

233    1.0
234    NaN
235    NaN
236    NaN
237    NaN
Name: label, dtype: float64

In [141]:
edittrain = kanye[:234]
edittest = kanye[234:]
droplabel = edittest.drop('label', axis = 1)

In [142]:
edittrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234 entries, 0 to 233
Columns: 861 entries, 100 to label
dtypes: float64(861)
memory usage: 1.5 MB


In [143]:
edittest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 234 to 283
Columns: 861 entries, 100 to label
dtypes: float64(861)
memory usage: 336.4 KB


In [144]:
x = edittrain.drop('label', axis = 1)
y = edittrain['label']

In [145]:
xtrain, xtest, ytrain, ytest = tts(x,y, random_state = 0)

In [146]:
l = LogisticRegression()
d = DecisionTreeClassifier()
r = RandomForestClassifier()
g = GradientBoostingClassifier()
k = KNeighborsClassifier()
nb = MultinomialNB()
vc = VotingClassifier(estimators = [('l',l),('d',d),('r',r),('g',g),('k',k),('nb',nb)],
                      voting = 'hard', n_jobs = 5)

In [147]:
def classifier(ye):
    print(ye)
    clf = ye.fit(xtrain,ytrain)
    clf.predict(droplabel)
    print('Accuracy of train: ', ye.score(xtrain,ytrain))
    print('Accuracy of test: ', ye.score(xtest,ytest))
    score = cvs(ye,xtest,ytest,cv=5, scoring='accuracy')
    print('cross val score: ', score.mean())

In [148]:
classifier(LogisticRegression())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Accuracy of train:  0.9942857142857143
Accuracy of test:  0.864406779661017
cross val score:  0.8503496503496504


In [149]:
classifier(DecisionTreeClassifier())

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Accuracy of train:  1.0
Accuracy of test:  0.7457627118644068
cross val score:  0.7958041958041958


In [150]:
classifier(RandomForestClassifier())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy of train:  0.9885714285714285
Accuracy of test:  0.7457627118644068
cross val score:  0.8475524475524475


In [151]:
classifier(GradientBoostingClassifier())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)
Accuracy of train:  1.0
Accuracy of test:  0.7627118644067796
cross val score:  0.813986013986014


In [152]:
classifier(KNeighborsClassifier())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
Accuracy of train:  0.88
Accuracy of test:  0.7966101694915254
cross val score:  0.8601398601398602


In [153]:
classifier(MultinomialNB())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Accuracy of train:  0.9885714285714285
Accuracy of test:  0.864406779661017
cross val score:  0.8503496503496504


In [154]:
classifier(vc)

VotingClassifier(estimators=[('l', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('d', DecisionT...
           weights='uniform')), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         flatten_transform=None, n_jobs=5, voting='hard', weights=None)
Accuracy of train:  0.9942857142857143
Accuracy of test:  0.8135593220338984


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site

cross val score:  0.813986013986014


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Extracting Ypred

In [155]:
kn = k.fit(xtrain,ytrain)
ypred = kn.predict(droplabel)
ypred

array([1., 1., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
       0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1.])

In [156]:
len(ypred)

50

In [157]:
len(toadd)

50

In [158]:
results = pd.DataFrame(columns=['text','label'])
results['text'] = toadd
results['label'] = ypred
results.to_csv('kanye.csv')